In [2]:
import tensorflow as tf
from tensorflow.contrib import slim
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import os
from PIL import Image
from collections import OrderedDict

%load_ext autoreload
%autoreload 2

In [3]:
z = np.ones((1, 100)) * 0.5#np.random.randn(1, 100)

In [4]:
raw_data = np.load("datasets/omniglot_data.npy")

In [440]:
# inp = np.ones((1, 28, 28, 1))
inp = np.array([raw_data[1200][0]], dtype="float64")

In [6]:
from dagan_architectures import UResNetGenerator, Discriminator

g = UResNetGenerator(batch_size=1, layer_sizes=[64, 64, 128, 128], layer_padding=None,
                        inner_layers=[3] * 4, name="generator")

In [7]:
z_inp = tf.placeholder(tf.float32, [1,100], 'z-inp')
cond_inp = tf.placeholder(tf.float32, [1,28,28,1], 'cond-inp')

In [356]:
result = g(z_inp, cond_inp, training=False)

0 [1, 2, 2, 128]
Tensor("generator_9/g_deconv_layers/concat:0", shape=(1, 2, 2, 136), dtype=float32)
Current Layers
Tensor("generator_9/g_deconv_layers/concat:0", shape=(1, 2, 2, 136), dtype=float32)


Tensor("generator_9/g_deconv_layers/concat:0", shape=(1, 2, 2, 136), dtype=float32)


Tensor("generator_9/g_deconv_layers/g_deconv0/BatchNorm/batchnorm_1/add_1:0", shape=(1, 2, 2, 128), dtype=float32)
Current Layers
Tensor("generator_9/g_deconv_layers/concat:0", shape=(1, 2, 2, 136), dtype=float32)


Tensor("generator_9/g_deconv_layers/g_deconv0/BatchNorm/batchnorm_1/add_1:0", shape=(1, 2, 2, 128), dtype=float32)
Tensor("generator_9/g_deconv_layers/concat:0", shape=(1, 2, 2, 136), dtype=float32)


Tensor("generator_9/g_deconv_layers/g_deconv0/BatchNorm_1/batchnorm/add_1:0", shape=(1, 2, 2, 128), dtype=float32)
Current Layers
Tensor("generator_9/g_deconv_layers/concat:0", shape=(1, 2, 2, 136), dtype=float32)


Tensor("generator_9/g_deconv_layers/g_deconv0/BatchNorm_1/batchnorm/add_1:0", s

Tensor("generator_9/g_deconv_layers/g_deconv3/BatchNorm_1/batchnorm/add_1:0", shape=(1, 14, 14, 64), dtype=float32)
Current Layers
Tensor("generator_9/g_deconv_layers/concat:0", shape=(1, 2, 2, 136), dtype=float32)
Tensor("generator_9/g_deconv_layers/g_deconv0/BatchNorm_2/batchnorm/add_1:0", shape=(1, 2, 2, 128), dtype=float32)
Tensor("generator_9/g_deconv_layers/concat_1:0", shape=(1, 4, 4, 260), dtype=float32)
Tensor("generator_9/g_deconv_layers/g_deconv1/BatchNorm_2/batchnorm/add_1:0", shape=(1, 4, 4, 128), dtype=float32)
Tensor("generator_9/g_deconv_layers/concat_2:0", shape=(1, 7, 7, 194), dtype=float32)
Tensor("generator_9/g_deconv_layers/g_deconv2/BatchNorm_2/batchnorm/add_1:0", shape=(1, 7, 7, 64), dtype=float32)
Tensor("generator_9/g_deconv_layers/g_deconv2/concat_4:0", shape=(1, 14, 14, 128), dtype=float32)


Tensor("generator_9/g_deconv_layers/g_deconv3/BatchNorm_1/batchnorm/add_1:0", shape=(1, 14, 14, 64), dtype=float32)
Tensor("generator_9/g_deconv_layers/g_deconv3/conv2d_

In [357]:
checkpoint = "checkpoints/checkpoint.ckpt"
fine_tune = slim.assign_from_checkpoint_fn(
    checkpoint,
    tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

sess = tf.Session()
fine_tune(sess)

INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt


In [441]:
final_results = sess.run(
    result,
    feed_dict={z_inp: z, cond_inp: inp},
)
final_results[0][0, :, :, 0][5][10]

0.9989437

In [428]:
class _SamePad(nn.Module):
    """
    Pads equivalent to the behavior of tensorflow "SAME"
    """
    def __init__(self, stride):
        super().__init__()
        self.stride = stride
    
    def forward(self, x):
        if self.stride == 2 and x.shape[2] % 2 == 0:
            return F.pad(x, (0, 1, 0, 1))
        return F.pad(x, (1, 1, 1, 1))

def _conv2d(in_channels, out_channels, kernel_size, stride, activate=True, dropout=0.0):
    layers = OrderedDict()
    layers['pad'] = _SamePad(stride)
    layers['conv'] = nn.Conv2d(in_channels, out_channels, kernel_size, stride)
    if activate:
        layers['relu'] = nn.LeakyReLU(0.2)
        layers['batchnorm'] = nn.BatchNorm2d(out_channels, eps=1e-3, momentum=0.01)
    
    if dropout > 0.0:
        layers['dropout'] = nn.Dropout(dropout)
    return nn.Sequential(layers)


def _conv2d_transpose(in_channels, out_channels, kernel_size, upscale_size, activate=True, dropout=0.0):
    layers = OrderedDict()
    layers['upsample'] = nn.Upsample(upscale_size)
    layers['conv'] = nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=1)
    if activate:
        layers['relu'] = nn.LeakyReLU(0.2)
        layers['batchnorm'] = nn.BatchNorm2d(out_channels, eps=1e-3, momentum=0.01)
    
    if dropout > 0.0:
        layers['dropout'] = nn.Dropout(dropout)
    return nn.Sequential(layers)


class _EncoderBlock(nn.Module):
    def __init__(
        self, pre_channels, in_channels, out_channels, num_layers, dropout_rate=0.0
    ):
        super().__init__()
        self.num_layers = num_layers
        self.pre_conv = _conv2d(
            in_channels=pre_channels, out_channels=pre_channels, kernel_size=3, stride=2, activate=False
        )

        self.conv0 = _conv2d(
            in_channels=in_channels + pre_channels, out_channels=out_channels, kernel_size=3, stride=1
        )
        total_channels = in_channels + out_channels
        for i in range(1, num_layers):
            self.add_module(
                'conv%d' % i,
                _conv2d(
                    in_channels=total_channels, out_channels=out_channels, kernel_size=3, stride=1
                )
            )
            total_channels += out_channels
        self.add_module(
            'conv%d' % num_layers,
            _conv2d(
                in_channels=total_channels, out_channels=out_channels, kernel_size=3, stride=2, dropout=dropout_rate
            )
        )

    def forward(self, inp):
        pre_input, x = inp
        pre_input = self.pre_conv(pre_input)
        out = self.conv0(torch.cat([x, pre_input], 1))
        
        all_outputs = [x, out]
        for i in range(1, self.num_layers + 1):
            input_features = torch.cat([all_outputs[-1], all_outputs[-2]] + all_outputs[:-2], 1)
            module = self._modules['conv%d' % i]
            out = module(input_features)
            all_outputs.append(out)
        return all_outputs[-2], all_outputs[-1]

    
class _DecoderBlock(nn.Module):
    def __init__(
        self, pre_channels, in_channels, out_channels, num_layers, curr_size, upscale_size=None, dropout_rate=0.0
    ):
        super().__init__()
        self.num_layers = num_layers
        self.should_upscale = upscale_size is not None
        self.should_pre_conv = pre_channels > 0
#         self.pre_conv = _conv2d(
#             in_channels=pre_channels, out_channels=pre_channels, kernel_size=3, stride=2, activate=False
#         )

#         self.conv0 = _conv2d(
#             in_channels=in_channels + pre_channels, out_channels=out_channels, kernel_size=3, stride=1
#         )

        total_channels = pre_channels + in_channels
        for i in range(num_layers):
            if self.should_pre_conv:
                self.add_module(
                    'pre_conv_t%d' % i,
                    _conv2d_transpose(
                        in_channels=pre_channels, out_channels=pre_channels,
                        kernel_size=3, upscale_size=curr_size, activate=False
                    )
                )
            self.add_module(
                'conv%d' % i,
                _conv2d(
                    in_channels=total_channels, out_channels=out_channels, kernel_size=3, stride=1
                )
            )
            total_channels += out_channels

        if self.should_upscale:
            total_channels -= pre_channels
            self.add_module(
                'conv_t%d' % num_layers,
                _conv2d_transpose(
                    in_channels=total_channels, out_channels=out_channels,
                    kernel_size=3, upscale_size=upscale_size, dropout=dropout_rate
                )
            )

    def forward(self, inp):
        pre_input, x = inp
#         pre_input = self.pre_conv(pre_input)
#         out = self.conv0(torch.cat([x, pre_input], 1))
#         print(pre_input.shape, x.shape)
        all_outputs = [x]
        for i in range(self.num_layers):
            curr_input = all_outputs[-1]
            if self.should_pre_conv:
                pre_conv_output = self._modules['pre_conv_t%d' % i](pre_input)
                curr_input = torch.cat([curr_input, pre_conv_output], 1)
            input_features = torch.cat([curr_input] + all_outputs[:-1], 1)
            module = self._modules['conv%d' % i]
            out = module(input_features)
            all_outputs.append(out)

        if self.should_upscale:
            module = self._modules['conv_t%d' % self.num_layers]
            input_features = torch.cat([all_outputs[-1], all_outputs[0]] + all_outputs[1:-1], 1)
            out = module(input_features)
            all_outputs.append(out)
        return all_outputs[-2], all_outputs[-1]


class TorchGAN(nn.Module):
    def __init__(self, dim, channels, dropout_rate=0.0, z_dim=100):
        super().__init__()
        self.dim = dim
        self.z_dim = z_dim
        self.channels = channels
        self.layer_sizes = [64, 64, 128, 128]
        self.num_inner_layers = 3

        # Number of times dimension is halved
        self.U_depth = len(self.layer_sizes)
        
        # dimension at each level of U-net
        self.dim_arr = [dim]
        for i in range(self.U_depth):
            self.dim_arr.append((self.dim_arr[-1] + 1) // 2)
        
        # Encoders
        self.encode0 = _conv2d(
            in_channels=1, out_channels=self.layer_sizes[0], kernel_size=3, stride=2
        )
        for i in range(1, self.U_depth):
            self.add_module(
                "encode%d" % i,
                _EncoderBlock(
                    pre_channels=self.layer_sizes[i - 1],
                    in_channels=self.layer_sizes[i - 1],
                    out_channels=self.layer_sizes[i],
                    num_layers=self.num_inner_layers,
                    dropout_rate=dropout_rate,
                )
            )
        
        # Noise encoders
        self.noise_encoders = 3
        num_noise_filters = 8
        self.z_channels = []
        for i in range(self.noise_encoders):
            curr_dim = self.dim_arr[-1 - i]  # Iterate dim from back
            self.add_module(
                "z_reshape%d" % i,
                nn.Linear(self.z_dim, curr_dim * curr_dim * num_noise_filters)
            )
            self.z_channels.append(num_noise_filters)
            num_noise_filters //= 2
            
        # Decoders
        for i in range(self.U_depth + 1):
            # Input from previous decoder
            in_channels = 0 if i == 0 else self.layer_sizes[-i]
            # Input from encoder across the "U"
            in_channels += self.channels if i == self.U_depth else self.layer_sizes[-i - 1]
            # Input from injected noise
            if i < self.noise_encoders:
                in_channels += self.z_channels[i]

            self.add_module(
                "decode%d" % i,
                _DecoderBlock(
                    pre_channels=0 if i == 0 else self.layer_sizes[-i],
                    in_channels=in_channels,
                    out_channels=self.layer_sizes[0] if i == self.U_depth else self.layer_sizes[-i - 1],
                    num_layers=self.num_inner_layers,
                    curr_size=self.dim_arr[-i - 1],
                    upscale_size=None if i == self.U_depth else self.dim_arr[-i - 2],
                    dropout_rate=dropout_rate,
                )
            )

        # Final conv
        self.num_final_conv = 3
        for i in range(self.num_final_conv - 1):
            self.add_module(
                "final_conv%d" % i,
                _conv2d(
                    in_channels=self.layer_sizes[0], out_channels=self.layer_sizes[0], kernel_size=3, stride=1
                )
            )
        self.add_module(
            "final_conv%d" % (self.num_final_conv - 1),
            _conv2d(
                in_channels=self.layer_sizes[0], out_channels=self.channels, kernel_size=3, stride=1, activate=False
            )
        )
        self.tanh = nn.Tanh()

    def forward(self, x, z):
        # Final output of every encoding block
        all_outputs = [x, self.encode0(x)]

        # Last 2 layer outputs
        out = [x, self.encode0(x)]
        for i in range(1, len(self.layer_sizes)):
            out = self._modules["encode%d" % i](out)
            all_outputs.append(out[1])
        
        print(out[1][0][0])
        pre_input, curr_input = None, out[1]
        for i in range(self.U_depth + 1):
            if i > 0:
                curr_input = torch.cat([curr_input, all_outputs[-i - 1]], 1)
            if i < self.noise_encoders:
                z_out = self._modules["z_reshape%d" % i](z)
                
                # TODO remove transform when removing tf dependency
                curr_dim = self.dim_arr[-i - 1]
                z_out = z_out.view(-1, curr_dim, curr_dim, self.z_channels[i])
                z_out = z_out.transpose(1, 3).transpose(2, 3)
                curr_input = torch.cat([z_out, curr_input], 1)
            
            pre_input, curr_input = self._modules["decode%d" % i]([pre_input, curr_input])
        
        print(curr_input[0][0][0])
        for i in range(self.num_final_conv):
            curr_input = self._modules["final_conv%d" % i](curr_input)
        return self.tanh(curr_input)

In [429]:
torch_g = TorchGAN(dim=28, channels=1)
torch_g.eval()

TorchGAN(
  (encode0): Sequential(
    (pad): _SamePad()
    (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2))
    (relu): LeakyReLU(negative_slope=0.2)
    (batchnorm): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (encode1): _EncoderBlock(
    (pre_conv): Sequential(
      (pad): _SamePad()
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
    )
    (conv0): Sequential(
      (pad): _SamePad()
      (conv): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
      (relu): LeakyReLU(negative_slope=0.2)
      (batchnorm): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    )
    (conv1): Sequential(
      (pad): _SamePad()
      (conv): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
      (relu): LeakyReLU(negative_slope=0.2)
      (batchnorm): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    )
    (conv2): Sequential(
      (pad): _SamePad()
      (conv

In [430]:
ckpt_reader = tf.train.load_checkpoint(checkpoint)
def read_tf(var):
    return torch.tensor(ckpt_reader.get_tensor(var))

def reshape_conv(tensor):
    return tensor.transpose(0, 3).transpose(1, 2).transpose(2, 3)

In [436]:
def convert_conv_layer(torch_layer, conv_path, bn_path=None):
    if bn_path is not None:
        torch_layer.batchnorm.weight.data = read_tf(bn_path + "/gamma")
        torch_layer.batchnorm.bias.data = read_tf(bn_path + "/beta")
        torch_layer.batchnorm.running_mean = read_tf(bn_path + "/moving_mean")
        torch_layer.batchnorm.running_var = read_tf(bn_path + "/moving_variance")
    
    torch_layer.conv.weight.data = reshape_conv(read_tf(conv_path + "/kernel"))
    torch_layer.conv.bias.data = read_tf(conv_path + "/bias")
    
num_layers = torch_g.num_inner_layers

# Encoders
convert_conv_layer(
    torch_g.encode0, "generator/conv_layers/g_conv0/conv2d", "generator/conv_layers/g_conv0/BatchNorm")

for i in range(1, torch_g.U_depth):
    module = torch_g._modules["encode%d" % i]
    convert_conv_layer(
        module.pre_conv,
        f"generator/conv_layers/g_conv{i}/conv2d"
    )

    for j in range(num_layers + 1):
        bn_suffix = f"_{j}" if j > 0 else ""
        convert_conv_layer(
            module._modules[f"conv{j}"],
            f"generator/conv_layers/g_conv{i}/conv2d_{j + 1}",
            f"generator/conv_layers/g_conv{i}/BatchNorm{bn_suffix}",
        )

# Noise encoders
for i in range(torch_g.noise_encoders):
    module = torch_g._modules["z_reshape%d" % i]
    z_suffix = f"_{i}" if i > 0 else ""
    module.weight.data = read_tf(f"generator/vector_expansion/dense{z_suffix}/kernel").T
    module.bias.data = read_tf(f"generator/vector_expansion/dense{z_suffix}/bias").T


# Decoders
for i in range(torch_g.U_depth + 1):
    module = torch_g._modules["decode%d" % i]

    for j in range(num_layers):
        idx_suffix = f"_{j}" if j > 0 else ""
        if i > 0:
            convert_conv_layer(
                module._modules[f"pre_conv_t{j}"],
                f"generator/g_deconv_layers/g_deconv{i}/conv2d_transpose{idx_suffix}",
            )
        convert_conv_layer(
            module._modules[f"conv{j}"],
            f"generator/g_deconv_layers/g_deconv{i}/conv2d{idx_suffix}",
            f"generator/g_deconv_layers/g_deconv{i}/BatchNorm{idx_suffix}",
        )
    if i < torch_g.U_depth:
        conv_suffix = f"_{num_layers}" if i > 0 else ""
        convert_conv_layer(
            module._modules[f"conv_t{num_layers}"],
            f"generator/g_deconv_layers/g_deconv{i}/conv2d_transpose{conv_suffix}",
            f"generator/g_deconv_layers/g_deconv{i}/BatchNorm_{num_layers}",
        )
        
# Final conv
for i in range(torch_g.num_final_conv):
    idx_suffix = f"_{i}" if i > 0 else ""
    if i == torch_g.num_final_conv - 1:
        bn_path = None
    else:
        bn_path = f"generator/g_deconv_layers/BatchNorm{idx_suffix}"
    convert_conv_layer(
        torch_g._modules[f"final_conv{i}"],
        f"generator/g_deconv_layers/conv2d{idx_suffix}",
        bn_path
    )

In [437]:
# read_tf(f"generator/vector_expansion/dense/kernel").T.shape

In [445]:
torch.tensor(inp).

torch.Size([1, 28, 28, 1])

In [448]:
# sum(sum(torch_g.conv.weight[0][0]))
# print(torch_g(torch.ones(1, 1, 28, 28)).shape)
# torch_inp = torch.ones(1, 1, 28, 28)
torch_inp = torch.tensor(inp).transpose(1, 3).transpose(2, 3).float()
torch_g(torch_inp, torch.tensor(z).float())[0][0][0]

tensor([[-0.5892, -0.4865],
        [ 0.5238, -0.4740]], grad_fn=<SelectBackward>)
tensor([ 0.0840, -0.0228, -0.0525, -0.0474, -0.0420, -0.0397, -0.0376, -0.0234,
        -0.0068,  0.0021, -0.0040, -0.0167, -0.0205, -0.0231, -0.0167, -0.0158,
         0.0006,  0.0117,  0.0174,  0.0188,  0.0079, -0.0050, -0.0161, -0.0455,
        -0.0516, -0.0863, -0.0326,  0.0853], grad_fn=<SelectBackward>)


tensor([0.9968, 0.9989, 0.9986, 0.9990, 0.9992, 0.9993, 0.9992, 0.9992, 0.9992,
        0.9992, 0.9992, 0.9992, 0.9992, 0.9991, 0.9990, 0.9989, 0.9990, 0.9990,
        0.9991, 0.9992, 0.9993, 0.9994, 0.9994, 0.9994, 0.9993, 0.9992, 0.9996,
        0.9983], grad_fn=<SelectBackward>)

In [442]:
final_results[0]

array([[[[ 0.9968462 ],
         [ 0.9989222 ],
         [ 0.99855083],
         [ 0.9990234 ],
         [ 0.99923205],
         [ 0.99925864],
         [ 0.999233  ],
         [ 0.99920547],
         [ 0.99920064],
         [ 0.99921155],
         [ 0.99922633],
         [ 0.9992216 ],
         [ 0.9991688 ],
         [ 0.99908626],
         [ 0.9989827 ],
         [ 0.9989365 ],
         [ 0.99896497],
         [ 0.99904543],
         [ 0.9991413 ],
         [ 0.99921304],
         [ 0.9992914 ],
         [ 0.99936825],
         [ 0.9993983 ],
         [ 0.999387  ],
         [ 0.99926925],
         [ 0.99922645],
         [ 0.9995618 ],
         [ 0.9982861 ]],

        [[ 0.99911857],
         [ 0.9995758 ],
         [ 0.9996732 ],
         [ 0.99970555],
         [ 0.9996501 ],
         [ 0.9996456 ],
         [ 0.9996344 ],
         [ 0.9996166 ],
         [ 0.99958926],
         [ 0.9996042 ],
         [ 0.99962914],
         [ 0.99967337],
         [ 0.9996983 ],
         [ 0.9

In [411]:
final_results[6][7][0, 0, :, 0]

array([-0.01885445, -0.09043905, -0.08474787, -0.09249571, -0.09400196,
       -0.09758668, -0.10315984, -0.11083569, -0.11560372, -0.11533213,
       -0.11428133, -0.10922816, -0.10963915, -0.11126538, -0.11048752,
       -0.10694195, -0.1028188 , -0.10580929, -0.10809049, -0.10763772,
       -0.10965155, -0.10563087, -0.10557331, -0.09269907, -0.09170057,
       -0.03120106, -0.01600707,  0.0479105 ], dtype=float32)

In [412]:
final_results[4][-1][0, 0, :, 0]

array([ 0.05900833, -0.09002143, -0.12935424, -0.1266709 , -0.1152868 ,
       -0.11829704, -0.12016031, -0.10493308, -0.09944692, -0.08946908,
       -0.08380413, -0.09614819, -0.09537685, -0.09238225, -0.07581463,
       -0.06762719, -0.06412295, -0.06768554, -0.07433248, -0.07418811,
       -0.07031867, -0.06839621, -0.06274238, -0.07325906, -0.06952679,
       -0.0997048 , -0.06428728,  0.07308565], dtype=float32)

In [168]:
final_results[3][13][0, :, :, 0]

array([[-0.5379593 , -0.33272868],
       [ 0.28598887, -0.30941498]], dtype=float32)

In [122]:
def render_image_arr(arr):
    arr = np.uint8(arr * 256)
    arr = arr.reshape(arr.shape[:-1])
    display(Image.fromarray(arr, mode='L').resize((224, 224)))

In [123]:
vars_list = tf.train.list_variables(checkpoint)
vars_list

[('beta1_power', []),
 ('beta1_power_1', []),
 ('beta2_power', []),
 ('beta2_power_1', []),
 ('discriminator/conv_layers/g_conv0/LayerNorm/beta', [64]),
 ('discriminator/conv_layers/g_conv0/LayerNorm/beta/Adam', [64]),
 ('discriminator/conv_layers/g_conv0/LayerNorm/beta/Adam_1', [64]),
 ('discriminator/conv_layers/g_conv0/LayerNorm/gamma', [64]),
 ('discriminator/conv_layers/g_conv0/LayerNorm/gamma/Adam', [64]),
 ('discriminator/conv_layers/g_conv0/LayerNorm/gamma/Adam_1', [64]),
 ('discriminator/conv_layers/g_conv0/conv2d/bias', [64]),
 ('discriminator/conv_layers/g_conv0/conv2d/bias/Adam', [64]),
 ('discriminator/conv_layers/g_conv0/conv2d/bias/Adam_1', [64]),
 ('discriminator/conv_layers/g_conv0/conv2d/kernel', [3, 3, 2, 64]),
 ('discriminator/conv_layers/g_conv0/conv2d/kernel/Adam', [3, 3, 2, 64]),
 ('discriminator/conv_layers/g_conv0/conv2d/kernel/Adam_1', [3, 3, 2, 64]),
 ('discriminator/conv_layers/g_conv1/LayerNorm/beta', [64]),
 ('discriminator/conv_layers/g_conv1/LayerNorm/be

In [193]:
reshape_conv(read_tf("generator/g_deconv_layers/g_deconv0/conv2d_transpose/kernel")).shape

torch.Size([520, 128, 3, 3])

In [191]:
torch_g.decode2.conv_t3.conv.weight.shape

torch.Size([386, 64, 3, 3])

In [124]:
torch_g.conv._modules['0']

ModuleAttributeError: 'TorchGAN' object has no attribute 'conv'

In [ ]:
render_image_arr(raw_data[1200][0])
render_image_arr(final_results[0][0])

In [ ]:
tf.image.resize_nearest_neighbor(inp, (None, None))